In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pylab as plt
%matplotlib inline

In [46]:
from astroquery.simbad import Simbad
from astropy.table import Table, vstack

def read_simbad_table(obj_name, **kwargs):
    """Read in Astroquery results from Simbad given object name (str)
    
    Parameter
    ---------
    obj_name : (str or list of str) name(s) of object
    
    kwargs (optional)
    ------
    votfields : (list) list of votable field keywords, from http://simbad.u-strasbg.fr/Pages/guide/sim-fscript.htx
                default is ['sptype', 'fluxdata(V)', 'velocity', 'distance']
    """
    customSimbad = Simbad()
    votfields = kwargs.pop('votfields', ['sptype', 'fluxdata(V)', 'velocity', 'distance'])
#    customSimbad.add_votable_fields('sptype')
#    customSimbad.add_votable_fields('fluxdata(V)')
#    customSimbad.add_votable_fields('velocity')
#    customSimbad.add_votable_fields('distance')
    for vf in votfields:
        customSimbad.add_votable_fields(vf)
    if type(obj_name) is str:
        result_table = customSimbad.query_object(obj_name)
    else:
        result_table = customSimbad.query_objects(obj_name)
    return result_table

TWOPI = 2*np.pi
def get_angular_separation(ra1, dec1, ra2, dec2):
    """Compute the angular separation in radians between 2 objects given positions in degrees.
    
    Parameters
    ----------
    ra1: (float) right ascension of object 1 in degrees
    dec1: (float) declination of object 1 in degrees
    ra2: (float) right ascension of object 2 in degrees
    dec2: (float) declination of object 2 in degrees"""
    assert ra1>=0. and ra1<=TWOPI and ra2>=0. and ra2<=TWOPI, "RA must be in [0, 2pi]: {} {}".format(ra1, ra2)
    assert dec1>=-np.pi/2. and dec1<=np.pi/2. and dec2>=-np.pi/2. and dec2<=np.pi/2., "DEC must be in [-pi/2, pi/2]: {} {}".format(dec1, dec2)
    cosA = np.sin(dec1)*np.sin(dec2) + np.cos(dec1)*np.cos(dec2)*np.cos(ra1-ra2)
    return np.arccos(cosA)

In [52]:
## read in the biased draw from Aki and make 5 unbiased draws from them
Ndraw=5
obj_names_all = np.loadtxt('inputs/luvoir-A_stars.txt', usecols=(0,), delimiter=',', skiprows=1).astype(int)
obj_names = np.random.choice(obj_names_all, size=Ndraw, replace=False)
print(obj_names)

## stack each Simbad query results vertically in Astropy Table 
#result = read_simbad_table("Hip"+str(obj_names[0]))
#for ii in range(1,len(obj_names)):
#    result = vstack([result, read_simbad_table("Hip"+str(obj_names[ii]))])
obj_names = np.core.defchararray.add("HIP", obj_names.astype(str))
result = read_simbad_table(obj_names)
print(result)

[40693 57757  8102 77760 61317]
 MAIN_ID        RA           DEC      ... Distance_method   Distance_bibcode 
             "h:m:s"       "d:m:s"    ...                                    
--------- ------------- ------------- ... --------------- -------------------
HD  69830 08 18 23.9469 -12 37 55.810 ...           paral 2018yCat.1345....0G
* bet Vir 11 50 41.7182 +01 45 52.991 ...           paral 2018yCat.1345....0G
* tau Cet 01 44 04.0834 -15 56 14.926 ...           paral 2018yCat.1345....0G
* chi Her 15 52 40.5411 +42 27 05.451 ...           paral 2018yCat.1345....0G
* bet CVn 12 33 44.5448 +41 21 26.924 ...           paral 2018yCat.1345....0G


In [25]:
result['MAIN_ID', 'SP_TYPE', 'FLUX_V', 'FLUX_ERROR_V', 'RVZ_RADVEL', 'Distance_distance']

MAIN_ID,SP_TYPE,FLUX_V,FLUX_ERROR_V,RVZ_RADVEL,Distance_distance
,,mag,,km / s,
object,object,float32,float32,float64,float64
HD 191408,K2.5V,5.32,0.000,-129.300,6.0159
* 61 UMa,G8V,5.34,--,-5.180,9.5794
HD 152391,G8.5Vk:,6.64,--,45.090,16.7959
* iot Per,G0V,4.05,0.050,49.220,10.5088
* b Aql,G7IVHdel1,5.16,--,-100.250,14.9593


In [53]:
## convert the Simbad RA/DEC from hh.mm.ss to degrees
from astropy import units as u
from astropy.coordinates import SkyCoord

c = SkyCoord(result['RA'], result['DEC'], unit=(u.hourangle, u.deg))

In [54]:
angular_separations = []
for ii in range(len(obj_names)-1):
    angsep = get_angular_separation(c.ra[ii].radian, c.dec[ii].radian, 
                                    c.ra[ii+1].radian, c.dec[ii+1].radian) * 180/np.pi
    angular_separations.append(angsep)
    print("Ang sep [deg] btw. {} and {} = {}".format(obj_names[ii], obj_names[ii+1], 
                                           angsep))

Ang sep [deg] btw. HIP40693 and HIP57757 = 54.6037919087
Ang sep [deg] btw. HIP57757 and HIP8102 = 148.687366256
Ang sep [deg] btw. HIP8102 and HIP77760 = 141.814945597
Ang sep [deg] btw. HIP77760 and HIP61317 = 36.4906974073


In [66]:
# to compute the slew time need to know how the instrument will slew:
# take shortest path vs. along 1 axis then another...?

def get_slew_times(sep, vel=10.):
    return

In [ ]:
# fn to compute the astrometric (proper motion + parallax) signal given a set of parameters

# fn to parse the set of parameters for fiducial targets

# fn to compute the integration time/epochs needed for astrometric signal

# fn to compute slew time

# fn to minimize slew time

